<a href="https://colab.research.google.com/github/cbsobral/ml-fies/blob/main/Module01_Models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 1 - Load Dataset

Import train_set and train_target

In [42]:
import pandas as pd

url_train = "https://drive.google.com/file/d/11s7p2dPEmumvg__DQnGYETFsZrfVgGKK/view?usp=sharing"
path_train = "https://drive.google.com/uc?export=download&id="+url_train.split("/")[-2]
train = pd.read_csv(path_train)
train.head()

(69395, 33)

In [48]:
train_set = train.drop("default", axis=1) # drop targets for training set
train_target = train["default"].copy()

### 2 - Pipeline

The pipeline contains functions that will be used to transform the dataset. For the numeric attributes, we employ median and stdscaler. 

In [49]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer


num_pipeline = Pipeline([
        ("num_imputer", SimpleImputer(strategy="median")),
        ("std_scaler", StandardScaler()),
    ])

In [50]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OrdinalEncoder

ord_pipeline = Pipeline([
        ("ord_imputer", SimpleImputer(strategy="most_frequent")),
        ("ord_encoder", OrdinalEncoder()),
    ])

In [51]:
ord_attribs = ['igc','date_contract'] # 2 attributes

num_attribs = ['family_income',   #17
               'personal_income',
               'high_school_endyear',
               'n_sem_course',
               'n_completed_sem',
               'sem_funded',
               'fam_size',
               'income_pc',
               'tuition_current',
               'inc_prop',
               'perc_requested',
               'loan_value_sem',
               'student_resource',
               'loan_value',
               'loan_limit',
               'total_debt',
               'age']
  

cat_attribs = ['semester_enroll',  #9
               'gender',
               'occupation', 
               'marital_status',
               'ethnicity', 
               'public_hs', 
               'state_course', 
               'degree', 
               'contract_phase']

In [52]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

full_pipeline = ColumnTransformer([
        ("num", num_pipeline, num_attribs),
        ("cat", OneHotEncoder(), cat_attribs),
        ("ord", ord_pipeline,ord_attribs)
        ])

In [53]:
train_prepared = full_pipeline.fit_transform(train_set)

### 3 - Classifiers



In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

logr = LogisticRegression(max_iter=1000, random_state=42, solver='lbfgs')
logr.fit(train_prepared, train_target)
(cross_val_score(logr, train_prepared, train_target, cv=3, scoring="roc_auc")).mean()

In [ ]:
from sklearn.tree import DecisionTreeClassifier 

dtc = DecisionTreeClassifier()
dtc = dtc.fit(train_prepared, train_target)
(cross_val_score(dtc, train_prepared, train_target, cv=3, scoring="roc_auc")).mean()

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier()
rf.fit(fies_prepared, fies_labels)
(cross_val_score(rf, fies_prepared, fies_labels, cv=3, scoring="roc_auc")).mean()

In [ ]:
from sklearn.svm import LinearSVC

svm_clf = LinearSVC(C=1, loss="hinge")
svm_clf.fit(fies_prepared, fies_labels)
(cross_val_score(svm_clf, fies_prepared, fies_labels, cv=3, scoring="roc_auc")).mean()

In [ ]:
from sklearn.neural_network import MLPClassifier

mlp = MLPClassifier(hidden_layer_sizes=(10, 10, 10), max_iter=500)
mlp.fit(fies_prepared, fies_labels.values.ravel())
(cross_val_score(mlp, fies_prepared, fies_labels, cv=3, scoring="roc_auc")).mean()